In [ ]:
#import quandl
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(1)
import sklearn.datasets
import pandas as pd
from sklearn import preprocessing

backwards_points = 75

In [ ]:
#DJIA_data = quandl.get("BCB/UDJIAD1", start_date="2001-12-31", end_date="2005-12-31")
DJIA_test_plot = pd.read_csv("DJIA_w_topic.csv")

In [ ]:
#plot_test = pd.DataFrame(DJIA_data)
#DJIA_data = DJIA_data.reset_index()
#DJIA_test_plot = DJIA_test_plot.sort_values(by="Date", ascending=True)
DJIA_test_plot["Time (Days)"] = np.arange(len(DJIA_test_plot))
DJIA_test_plot = DJIA_test_plot.drop("Date", axis = 1)
DJIA_test_plot = DJIA_test_plot.drop("Label", axis = 1)
DJIA_test_plot.head()

In [ ]:
plt.plot(DJIA_test_plot["Time (Days)"], DJIA_test_plot["Open"], label='DJIA')

In [ ]:
def convert_data(your_set):
    DJIA_df = pd.read_csv("DJIA_w_topic.csv")
    #DJIA_df = DJIA_df.sort_values(by="Date", ascending=True)
    DJIA_df = DJIA_df.drop("Date", axis = 1)
    #DJIA_df = DJIA_df.drop("Label", axis = 1)
    DJIA_df = DJIA_df.values

    DJIA_df_normaliser = preprocessing.MinMaxScaler()
    DJIA_df_normalised = DJIA_df_normaliser.fit_transform(DJIA_df)
    past_values_normalised =  np.array([DJIA_df_normalised[i:i + backwards_points].copy() for i in range(len(DJIA_df_normalised) - backwards_points)])
    next_day_values_normalised =  np.array([DJIA_df_normalised[:,0][i + backwards_points].copy() for i in range(len(DJIA_df_normalised) - backwards_points)])
    next_day_values_normalised = np.expand_dims(next_day_values_normalised, -1)
    
    next_day_values = np.array([DJIA_df[:,0][i + backwards_points].copy() for i in range(len(DJIA_df) - backwards_points)])
    next_day_values = np.expand_dims(next_day_values, -1)
    y_normaliser = preprocessing.MinMaxScaler()
    y_normaliser.fit(next_day_values)
    
    return past_values_normalised, next_day_values_normalised, next_day_values, y_normaliser

In [ ]:
past_values, next_day_values, y_no_mod, y_normaliser = convert_data("DJIA_w_topic.csv")

test_split = 0.8
n = int(past_values.shape[0] * test_split)

X_train = past_values[:n]
y_train = next_day_values[:n]
X_test = past_values[n:]
y_test = next_day_values[n:]

y_no_mod_test = y_no_mod[n:]

print(X_train.shape)
print(X_test.shape)

In [ ]:
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dropout, Dense, Activation
from keras import optimizers

#https://keras.io/api/layers/recurrent_layers/lstm/

lstm_input = Input(shape = (backwards_points, 8))
x = LSTM(75)(lstm_input)
x = Dropout(0.21)(x)
x = Dense(64)(x)
x = Activation("sigmoid")(x)
x = Dense(1)(x)
output = Activation("linear")(x)

model = Model(inputs = lstm_input, outputs=output)
model.compile(optimizer = "adam", loss="mse")
model.fit(X_train, y_train, batch_size=32, epochs=50, shuffle=True, validation_split=0.2)
print(model.summary())

In [ ]:
y_test_predicted = model.predict(X_test)
y_test_predicted = y_normaliser.inverse_transform(y_test_predicted)
y_predicted = model.predict(past_values)
y_predicted = y_normaliser.inverse_transform(y_predicted)

#https://www.geeksforgeeks.org/python-mean-squared-error/
MSE = np.square(np.subtract(y_no_mod_test, y_test_predicted)).mean() 
returned_mse = MSE / (np.max(y_no_mod_test) - np.min(y_no_mod_test)) * 100
print(returned_mse)

In [ ]:
#get current figure
plt.gcf().set_size_inches(25, 15, forward=True)
plt.plot(y_no_mod_test, label="Real", color="blue")
plt.plot(y_test_predicted, label="Prediction", color="red")
plt.legend(["Realz", "Predicted"])
plt.title("LSTM Topics Model")

#plt.savefig("LSTM_w_Topic_0.png")
plt.show()

In [ ]:
plt.gcf().set_size_inches(25, 15, forward=True)
plt.plot(y_no_mod, label="Real", color="blue")
plt.plot(y_predicted, label="Prediction", color="red")
plt.legend(["Realz", "Predicted"])
plt.title("LSTM Topics Model")

#plt.savefig("LSTM_w_Topic_1.png")
plt.show()